In [56]:
# Dependencies
import pandas as pd
import numpy as np
import time

#Load csv data into dataframes
mh_2015_full = pd.read_csv("Resources/2015/Data/2015_mh_data.csv")
mh_2016_full = pd.read_csv("Resources/2016/Data/2016_mh_data.csv")
mh_2017_full = pd.read_csv("Resources/2017/Data/2017_mh_data.csv")
mh_2018_full = pd.read_csv("Resources/2018/Data/2018_mh_data.csv")
mh_suicide = pd.read_csv("Resources/suicide_mortality _multiyear.csv")
election2016_data =  pd.read_csv("Resources/2016/data/election2016.csv")



In [6]:
gov_election2016_data = election2016_data.copy()
gov_election2016_data = gov_election2016_data.iloc[:56,:4] 
convert_to_number_list = ["Democratic Candidates","Republican Candidates","Other Candidates"]
for item in ["Democratic Candidates","Republican Candidates","Other Candidates"]:
    for x in range(len(gov_election2016_data[item])):
        gov_election2016_data[item][x] = int(gov_election2016_data[item][x].replace(',' , ''))
        
#Citing the website that provided a dictionary of state abreviations and state names
#""" Title: PYTHON DICTIONARY OF US STATES AND TERRITORIES (PYTHON RECIPE)
#Author: Mike Shultz 
#Date: July 14, 2010
#Code version: 1
#Availability: http://code.activestate.com/recipes/577305-python-dictionary-of-us-states-and-territories/ """

#Dictionary of state abreviations

us_states = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MP': 'Northern Mariana Islands',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming',
        'ZZ': 'Unknown'
}
gov_election2016_data["Party"] = gov_election2016_data["State"]
for state in range(len(gov_election2016_data["State"])):
    try:
        if gov_election2016_data.iloc[state,1] > gov_election2016_data.iloc[state,2]:
            if gov_election2016_data.iloc[state,1] > gov_election2016_data.iloc[state,3]:
                gov_election2016_data.iloc[state, 4] = "D"
        elif gov_election2016_data.iloc[state,2] > gov_election2016_data.iloc[state,1]:
            if gov_election2016_data.iloc[state,2] > gov_election2016_data.iloc[state,3]:
                gov_election2016_data.iloc[state, 4] = "R"
        else:
            gov_election2016_data.iloc[state, 4] = "I"
        gov_election2016_data["State"][state] = us_states[gov_election2016_data["State"][state]]
    except:
        None
gov_party = gov_election2016_data[["State","Party"]]
gov_party.to_csv("output/gov_election2016.csv", index=False, header=True)
gov_party.head()

,State,Party
0,Alabama,R
1,Alaska,R
2,American Samoa,R
3,Arizona,R
4,Arkansas,R


In [31]:
#Rename inconsistent column in 2017 dataset
mh_2017_full =mh_2017_full.rename(columns = {'lst':'LST'})

#Create a list of mh dataframes
dataset_lst = [mh_2015_full, mh_2016_full, mh_2017_full, mh_2018_full]


#Loops through each mh dataframe and adds the full state name into a new column state
for dataset in dataset_lst:

    index_count = 0
    dataset['state'] = ""
    
    for rows in dataset.iterrows():
        abr_name = dataset.loc[index_count, 'LST']
        full_name = us_states[abr_name]
        dataset.loc[index_count, 'state'] = full_name
        index_count = index_count + 1


In [33]:
#Loops through suicide dataframe and adds the full state name into a new column state
index_count = 0
mh_suicide['state'] = ""

for rows in mh_suicide.iterrows():
        abr_name = mh_suicide.loc[index_count, 'STATE']
        full_name = us_states[abr_name]
        mh_suicide.loc[index_count, 'state'] = full_name
        index_count = index_count + 1
        


In [34]:
#Break up suicde data by year. Create a dataframe for each. 
mh_2017_suicide = mh_suicide.loc[mh_suicide['YEAR']== 2017]
mh_2017_suicide= mh_2017_suicide[['state', 'RATE']]

mh_2017_suicide.to_csv('Output/mh_suicide_2017.csv')

mh_2016_suicide = mh_suicide.loc[mh_suicide['YEAR']== 2016]
mh_2016_suicide = mh_2017_suicide[['state', 'RATE']]

mh_2015_suicide = mh_suicide.loc[mh_suicide['YEAR']== 2015]
mh_2015_suicide = mh_2015_suicide[['state', 'RATE']]

#Merge each year of suicide data into one so that each year has a seperate column

suicide_rates= pd.merge(mh_2015_suicide, mh_2016_suicide, on='state', suffixes = ('_2015', '_2016') )
suicide_rates_all = pd.merge(suicide_rates, mh_2017_suicide, on='state' )
suicide_rates_all = suicide_rates_all.rename(columns ={'RATE_2015': '2015','RATE_2016': '2016', 'RATE': '2017'})

suicide_rates_all.to_csv('Output/mh_suicide_all.csv')

bottom5_suicide = mh_2017_suicide.nsmallest(5, 'RATE')
top5_suicide = mh_2017_suicide.nlargest(5, 'RATE')

bottom5_names = bottom5_suicide['state']
top5_names = top5_suicide['state']
bottom5_suicide

,state,RATE
31,New York,8.1
29,New Jersey,8.3
20,Massachusetts,9.5
19,Maryland,9.8
4,California,10.5


In [35]:
top5_suicide

,state,RATE
25,Montana,28.9
1,Alaska,27.0
49,Wyoming,26.9
30,New Mexico,23.3
11,Idaho,23.2


In [36]:
#Create a list of colomn names for condensed dataframes
mh_cols = ['state', 'FOCUS', 'CRISISTEAM2', 'TREATMT', 'MHDIAGEVAL', 'MHREFERRAL', 'MHEMGCY', 'MHSUICIDE', 'PAYASST',
           'LANG', 'LANG16', 'CHILDAD', 'ADOLES', 'YOUNGADULTS', 'ADULT', 'SENIORS', 'SRVC62', 'SRVC113', 'SRVC31', 'MHHOUSING', 'MHEDUCATION']
#Create condensed dataframes
mh_2015_dem = mh_2015_full[mh_cols]
mh_2016_dem = mh_2016_full[mh_cols]
mh_2017_dem = mh_2017_full[mh_cols]
mh_2018_dem = mh_2018_full[mh_cols]

dataset_lst_dem = [mh_2015_dem, mh_2016_dem, mh_2017_dem, mh_2018_dem]

#Create a list of only integer column names
mh_cols_int = ['FOCUS', 'CRISISTEAM2', 'TREATMT', 'MHDIAGEVAL', 'MHREFERRAL', 'MHEMGCY', 'MHSUICIDE', 'PAYASST',
           'LANG', 'LANG16', 'CHILDAD', 'ADOLES', 'YOUNGADULTS', 'ADULT', 'SENIORS', 'SRVC62', 'SRVC113', 'SRVC31', 'MHHOUSING', 'MHEDUCATION']

#Loop through datasets, rows, and columns to replace all negative values with zeros. Survey records a question skip as a negative value.
index_count = 0

for dataset in dataset_lst_dem:

    index_count = 0
    
    for rows in dataset.iterrows():
        
        for col in mh_cols_int:
            
            if dataset.loc[index_count, col] < 0:
            
                dataset.loc[index_count, col] = 0
        
        index_count = index_count + 1
     

In [38]:
#Group each dataframe by state. Count the focus column to get a count of facilities. 
#Sum the rest of the columns since y/n is represented by 1/0
mh_2015_dem_group = mh_2015_dem.groupby(['state']).agg({'FOCUS': 'count', 'CRISISTEAM2': 'sum', 'TREATMT': 'sum', 'MHDIAGEVAL': 'sum',
                                    'MHREFERRAL': 'sum', 'MHEMGCY': 'sum', 'MHSUICIDE': 'sum', 'PAYASST': 'sum',
                                    'LANG': 'sum', 'LANG16': 'sum', 'CHILDAD': 'sum', 'ADOLES': 'sum', 'YOUNGADULTS': 'sum',
                                    'ADULT': 'sum', 'SENIORS': 'sum', 'SRVC62': 'sum', 'SRVC113': 'sum', 'SRVC31': 'sum', 'MHHOUSING': 'sum', 'MHEDUCATION': 'sum' })

mh_2016_dem_group = mh_2016_dem.groupby(['state']).agg({'FOCUS': 'count', 'CRISISTEAM2': 'sum', 'TREATMT': 'sum', 'MHDIAGEVAL': 'sum',
                                    'MHREFERRAL': 'sum', 'MHEMGCY': 'sum', 'MHSUICIDE': 'sum', 'PAYASST': 'sum',
                                    'LANG': 'sum', 'LANG16': 'sum', 'CHILDAD': 'sum', 'ADOLES': 'sum', 'YOUNGADULTS': 'sum',
                                    'ADULT': 'sum', 'SENIORS': 'sum', 'SRVC62': 'sum', 'SRVC113': 'sum', 'SRVC31': 'sum' , 'MHHOUSING': 'sum', 'MHEDUCATION': 'sum'})

mh_2017_dem_group = mh_2017_dem.groupby(['state']).agg({'FOCUS': 'count', 'CRISISTEAM2': 'sum', 'TREATMT': 'sum', 'MHDIAGEVAL': 'sum',
                                    'MHREFERRAL': 'sum', 'MHEMGCY': 'sum', 'MHSUICIDE': 'sum', 'PAYASST': 'sum',
                                    'LANG': 'sum', 'LANG16': 'sum', 'CHILDAD': 'sum', 'ADOLES': 'sum', 'YOUNGADULTS': 'sum',
                                    'ADULT': 'sum', 'SENIORS': 'sum', 'SRVC62': 'sum', 'SRVC113': 'sum', 'SRVC31': 'sum' , 'MHHOUSING': 'sum', 'MHEDUCATION': 'sum'})

mh_2018_dem_group = mh_2018_dem.groupby(['state']).agg({'FOCUS': 'count', 'CRISISTEAM2': 'sum', 'TREATMT': 'sum', 'MHDIAGEVAL': 'sum',
                                    'MHREFERRAL': 'sum', 'MHEMGCY': 'sum', 'MHSUICIDE': 'sum', 'PAYASST': 'sum',
                                    'LANG': 'sum', 'LANG16': 'sum', 'CHILDAD': 'sum', 'ADOLES': 'sum', 'YOUNGADULTS': 'sum',
                                    'ADULT': 'sum', 'SENIORS': 'sum', 'SRVC62': 'sum', 'SRVC113': 'sum', 'SRVC31': 'sum', 'MHHOUSING': 'sum', 'MHEDUCATION': 'sum' })

#Rename column to reflect what it is measuring
mh_2015_dem_group =mh_2015_dem_group.rename(columns = {'FOCUS':'Num_Facilities', 'LANG16': 'LANG_SPAN', 'SRVC62': 'LGBT', 'SRVC113': 'Veterans', 'SRVC31': 'CoSubstance'})
mh_2016_dem_group =mh_2016_dem_group.rename(columns = {'FOCUS':'Num_Facilities', 'LANG16': 'LANG_SPAN', 'SRVC62': 'LGBT', 'SRVC113': 'Veterans', 'SRVC31': 'CoSubstance'})
mh_2017_dem_group =mh_2017_dem_group.rename(columns = {'FOCUS':'Num_Facilities', 'LANG16': 'LANG_SPAN', 'SRVC62': 'LGBT', 'SRVC113': 'Veterans', 'SRVC31': 'CoSubstance'})
mh_2018_dem_group =mh_2018_dem_group.rename(columns = {'FOCUS':'Num_Facilities', 'LANG16': 'LANG_SPAN', 'SRVC62': 'LGBT', 'SRVC113': 'Veterans', 'SRVC31': 'CoSubstance'})

In [40]:
#Merging 2015 - 2018 mh dataframes through outer merges

#Merge 2015 and 2016
merge_15_16 = pd.merge(mh_2015_dem_group, mh_2016_dem_group, on='state', suffixes = ('_2015', '_2016') )

#Merge the merged 2015/2016 and 2017
merge_15_16_17 = pd.merge(merge_15_16, mh_2017_dem_group, on='state' )

#Rename the columns from the 2017 dataframe
merge_15_16_17 = merge_15_16_17.rename(columns ={'Num_Facilities': 'Num_Facilities_2017', 'CRISISTEAM2':'CRISISTEAM2_2017' , 'TREATMT': 'TREATMT_2017',
                      'MHDIAGEVAL': 'MHDIAGEVAL_2017', 'MHREFERRAL': 'MHREFERRAL_2017', 'MHEMGCY': 'MHEMGCY_2017',
                      'MHSUICIDE': 'MHSUICIDE_2017', 'PAYASST': 'PAYASST_2017','LANG': 'LANG_2017','LANG_SPAN': 'LANG_SPAN_2017',
                     'CHILDAD': 'CHILDAD_2017', 'ADOLES': 'ADOLES_2017', 'YOUNGADULTS': 'YOUNGADULTS_2017', 'ADULT': 'ADULT_2017',
                      'SENIORS': 'SENIORS_2017', 'LGBT': 'LGBT_2017', 'Veterans' : 'Veterans_2017','CoSubstance': 'CoSubstance_2017', 'MHHOUSING': 'MHHOUSING_2017', 'MHEDUCATION': 'MHEDUCATION_2017' })

#Merge the merged 2015/2016/2017 and 2018
merge_15_16_17_18 = pd.merge(merge_15_16_17, mh_2018_dem_group, on='state' )

#Rename the columns from the 2018 dataframe
merge_15_16_17_18 = merge_15_16_17_18.rename(columns ={'Num_Facilities': 'Num_Facilities_2018', 'CRISISTEAM2':'CRISISTEAM2_2018' , 'TREATMT': 'TREATMT_2018',
                      'MHDIAGEVAL': 'MHDIAGEVAL_2018', 'MHREFERRAL': 'MHREFERRAL_2018', 'MHEMGCY': 'MHEMGCY_2018',
                      'MHSUICIDE': 'MHSUICIDE_2018', 'PAYASST': 'PAYASST_2018','LANG': 'LANG_2018','LANG_SPAN': 'LANG_SPAN_2018',
                     'CHILDAD': 'CHILDAD_2018', 'ADOLES': 'ADOLES_2018', 'YOUNGADULTS': 'YOUNGADULTS_2018', 'ADULT': 'ADULT_2018',
                      'SENIORS': 'SENIORS_2018', 'LGBT': 'LGBT_2018', 'Veterans' : 'Veterans_2018','CoSubstance': 'CoSubstance_2018', 'MHHOUSING': 'MHHOUSING_2018', 'MHEDUCATION': 'MHEDUCATION_2018'})

gov_party = gov_party.rename(columns ={'State':'state'})

merge_party = pd.merge(merge_15_16_17_18, gov_party, on='state')
merge_party = merge_party.set_index('state')
merge_party = merge_party.drop(['Puerto Rico'])
merge_party

,Num_Facilities_2015,CRISISTEAM2_2015,TREATMT_2015,MHDIAGEVAL_2015,MHREFERRAL_2015,MHEMGCY_2015,MHSUICIDE_2015,PAYASST_2015,LANG_2015,LANG_SPAN_2015,...,ADOLES_2018,YOUNGADULTS_2018,ADULT_2018,SENIORS_2018,LGBT_2018,Veterans_2018,CoSubstance_2018,MHHOUSING_2018,MHEDUCATION_2018,Party
state,,,,,,,,,,,,,,,,,,,,,
Alabama,197,103,64,164,163,61,98,125,104,21,...,83,143,152,150,17,29,58,20,60,R
Alaska,103,61,72,92,86,36,58,61,34,11,...,77,73,62,59,14,15,45,19,40,R
Arizona,386,125,287,281,246,87,168,129,283,244,...,194,319,310,302,92,47,194,73,127,R
Arkansas,259,206,96,249,243,109,129,141,157,34,...,158,184,180,168,33,22,91,17,63,R
California,913,438,421,831,771,198,420,483,751,675,...,459,736,616,520,215,95,407,260,223,D
Colorado,203,146,147,178,169,65,124,120,142,101,...,130,161,153,146,28,22,104,70,67,D
Connecticut,241,86,167,214,183,46,74,123,142,115,...,142,187,183,170,73,27,134,38,61,D
Delaware,31,15,18,28,25,11,14,19,22,14,...,24,34,34,28,9,10,17,5,13,D
District of Columbia,38,15,20,35,28,8,15,19,26,19,...,23,33,30,30,13,6,20,10,8,D


In [41]:
#MH Facilities offering LGBT Programs
mh_lgbt = merge_party[['Num_Facilities_2015','LGBT_2015','Num_Facilities_2016','LGBT_2016', 'Num_Facilities_2017','LGBT_2017', 'Num_Facilities_2018','LGBT_2018', 'Party']]

#Create columns that divide facilities with LGBT programs by total number of facilities for each year
mh_lgbt.loc[:,'2015'] = (mh_lgbt['LGBT_2015']/mh_lgbt['Num_Facilities_2015'])*100
mh_lgbt.loc[:,'2016'] = (mh_lgbt['LGBT_2016']/mh_lgbt['Num_Facilities_2016'])*100
mh_lgbt.loc[:,'2017'] = (mh_lgbt['LGBT_2017']/mh_lgbt['Num_Facilities_2017'])*100
mh_lgbt.loc[:,'2018'] = (mh_lgbt['LGBT_2018']/mh_lgbt['Num_Facilities_2018'])*100

mh_lgbt_trends = mh_lgbt[['2015','2016', '2017', '2018', 'Party']]
mh_lgbt_trends = mh_lgbt_trends.sort_values('2016', ascending=True)

#Export dataframe to csv
mh_lgbt_trends.to_csv('Output/mh_lgbt_trends.csv')
mh_lgbt_trends.head()

C:\Users\allym\Anaconda3\lib\site-packages\pandas\core\indexing.py:376: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)


,2015,2016,2017,2018,Party
state,,,,,
Iowa,6.024096,3.225806,7.333333,12.765957,R
South Dakota,16.666667,4.166667,3.921569,6.122449,R
Missouri,19.747899,4.566210,6.603774,9.952607,R
Arkansas,7.722008,4.680851,9.174312,14.537445,R
Kansas,7.692308,5.882353,10.569106,12.396694,R


In [42]:
#MH Facilities offering Veteran Programs
mh_veterans = merge_party[['Num_Facilities_2015','Veterans_2015','Num_Facilities_2016','Veterans_2016', 'Num_Facilities_2017','Veterans_2017', 'Num_Facilities_2018','Veterans_2018', 'Party' ]]

#Create columns that divide facilities with Veteran programs by total number of facilities for each year
mh_veterans.loc[:,'2015'] = (mh_veterans['Veterans_2015']/mh_veterans['Num_Facilities_2015'])*100
mh_veterans.loc[:,'2016'] = (mh_veterans['Veterans_2016']/mh_veterans['Num_Facilities_2016'])*100
mh_veterans.loc[:,'2017'] = (mh_veterans['Veterans_2017']/mh_veterans['Num_Facilities_2017'])*100
mh_veterans.loc[:,'2018'] = (mh_veterans['Veterans_2018']/mh_veterans['Num_Facilities_2018'])*100

mh_veteran_trends = mh_veterans[['2015','2016', '2017', '2018', 'Party']]
mh_veteran_trends = mh_veteran_trends.sort_values('2016', ascending=True)
#Export dataframe to csv
mh_veteran_trends.to_csv('Output/mh_verteran_trends.csv')
mh_veteran_trends.head()

,2015,2016,2017,2018,Party
state,,,,,
North Dakota,6.060606,3.125000,3.125000,8.823529,R
Arkansas,5.405405,3.829787,5.504587,9.691630,R
Iowa,7.831325,5.161290,10.000000,14.184397,R
Missouri,24.369748,7.762557,10.849057,14.691943,R
West Virginia,15.322581,7.964602,10.526316,12.396694,R


In [43]:
#MH Facilities offering housing Programs
mh_housing = merge_party[['Num_Facilities_2015','MHHOUSING_2015','Num_Facilities_2016','MHHOUSING_2016', 'Num_Facilities_2017','MHHOUSING_2017', 'Num_Facilities_2018','MHHOUSING_2018', 'Party' ]]

#Create columns that divide facilities with housing programs by total number of facilities for each year
mh_housing.loc[:,'2015'] = (mh_housing['MHHOUSING_2015']/mh_housing['Num_Facilities_2015'])*100
mh_housing.loc[:,'2016'] = (mh_housing['MHHOUSING_2016']/mh_housing['Num_Facilities_2016'])*100
mh_housing.loc[:,'2017'] = (mh_housing['MHHOUSING_2017']/mh_housing['Num_Facilities_2017'])*100
mh_housing.loc[:,'2018'] = (mh_housing['MHHOUSING_2018']/mh_housing['Num_Facilities_2018'])*100

mh_housing_trends = mh_housing[['2015','2016', '2017', '2018', 'Party']]
mh_housing_trends = mh_housing_trends.sort_values('2016', ascending=True)

#Export dataframe to csv
mh_housing_trends.to_csv('Output/mh_housing_trends.csv')
mh_housing_trends.head()

,2015,2016,2017,2018,Party
state,,,,,
Arkansas,5.405405,5.531915,7.798165,7.488987,R
Wisconsin,8.425721,9.302326,10.952381,10.617284,R
North Dakota,9.090909,9.375000,9.375000,11.764706,R
Iowa,9.638554,10.967742,9.333333,7.801418,R
Idaho,11.055276,12.429379,13.698630,11.811024,R


In [44]:
#MH Facilities offering education Programs
mh_education = merge_party[['Num_Facilities_2015','MHEDUCATION_2015','Num_Facilities_2016','MHEDUCATION_2016', 'Num_Facilities_2017','MHEDUCATION_2017', 'Num_Facilities_2018','MHEDUCATION_2018', 'Party' ]]

#Create columns that divide facilities with education programs by total number of facilities for each year
mh_education.loc[:,'2015'] = (mh_education['MHEDUCATION_2015']/mh_education['Num_Facilities_2015'])*100
mh_education.loc[:,'2016'] = (mh_education['MHEDUCATION_2016']/mh_education['Num_Facilities_2016'])*100
mh_education.loc[:,'2017'] = (mh_education['MHEDUCATION_2017']/mh_education['Num_Facilities_2017'])*100
mh_education.loc[:,'2018'] = (mh_education['MHEDUCATION_2018']/mh_education['Num_Facilities_2018'])*100

mh_education_trends = mh_education[['2015','2016', '2017', '2018', 'Party']]
mh_education_trends = mh_education_trends.sort_values('2016', ascending=True)

#Export dataframe to csv
mh_education_trends.to_csv('Output/mh_education_trends.csv')
mh_education_trends.head()

,2015,2016,2017,2018,Party
state,,,,,
Maine,18.433180,18.226601,22.043011,20.555556,D
Washington,18.045113,18.374558,19.558360,19.943820,D
District of Columbia,28.947368,19.512195,21.621622,22.857143,D
Connecticut,20.746888,21.888412,23.611111,26.872247,D
Maryland,21.967213,22.945205,23.106061,25.101215,D


In [45]:
#MH Facilities offering substance Programs
mh_substance = merge_party[['Num_Facilities_2015','TREATMT_2015','Num_Facilities_2016','TREATMT_2016', 'Num_Facilities_2017','TREATMT_2017', 'Num_Facilities_2018','TREATMT_2018', 'Party' ]]

#Create columns that divide facilities with substance programs by total number of facilities for each year
mh_substance.loc[:,'2015'] = (mh_substance['TREATMT_2015']/mh_substance['Num_Facilities_2015'])*100
mh_substance.loc[:,'2016'] = (mh_substance['TREATMT_2016']/mh_substance['Num_Facilities_2016'])*100
mh_substance.loc[:,'2017'] = (mh_substance['TREATMT_2017']/mh_substance['Num_Facilities_2017'])*100
mh_substance.loc[:,'2018'] = (mh_substance['TREATMT_2018']/mh_substance['Num_Facilities_2018'])*100

mh_substance_trends = mh_substance[['2015','2016', '2017', '2018', 'Party']]
mh_substance_trends = mh_substance_trends.sort_values('2016', ascending=True)

#Export dataframe to csv
mh_substance_trends.to_csv('Output/mh_substance_trends.csv')
mh_substance_trends.head()

,2015,2016,2017,2018,Party
state,,,,,
Pennsylvania,34.477124,33.959044,35.901926,35.294118,R
Arkansas,37.065637,34.468085,38.990826,40.088106,R
Alabama,32.487310,34.715026,33.142857,33.333333,R
Iowa,40.963855,34.838710,40.000000,41.134752,R
Minnesota,34.979424,36.250000,38.839286,40.000000,D


In [46]:
#MH Facilities offering suicide prevention services
mh_suicide = merge_party[['Num_Facilities_2015','MHSUICIDE_2015','Num_Facilities_2016','MHSUICIDE_2016', 'Num_Facilities_2017','MHSUICIDE_2017', 'Num_Facilities_2018','MHSUICIDE_2018', 'Party' ]]

#Create columns that divide facilities serving suicide prevention by total number of facilities for each year
mh_suicide.loc[:,'2015'] = (mh_suicide['MHSUICIDE_2015']/mh_suicide['Num_Facilities_2015'])*100
mh_suicide.loc[:,'2016'] = (mh_suicide['MHSUICIDE_2016']/mh_suicide['Num_Facilities_2016'])*100
mh_suicide.loc[:,'2017'] = (mh_suicide['MHSUICIDE_2017']/mh_suicide['Num_Facilities_2017'])*100
mh_suicide.loc[:,'2018'] = (mh_suicide['MHSUICIDE_2018']/mh_suicide['Num_Facilities_2018'])*100

mh_suicide_trends = mh_suicide[['2015','2016', '2017', '2018', 'Party']]
mh_suicide_trends = mh_suicide_trends.sort_values('2016', ascending=True)


#Export dataframe to csv
mh_suicide_trends.to_csv('Output/mh_suicide_trends.csv')
mh_suicide_trends.head()

,2015,2016,2017,2018,Party
state,,,,,
Maine,29.032258,31.034483,33.333333,37.222222,D
Massachusetts,34.117647,35.693215,36.825397,42.622951,D
Iowa,36.746988,38.064516,40.000000,44.680851,R
North Carolina,31.286550,38.486842,38.521401,38.675958,R
Ohio,34.974533,39.547038,42.468240,45.293073,R


In [47]:
#Merge Data on MH facility mental health services with suicide rate data
suicide_rate_service= pd.merge(mh_suicide_trends, mh_2017_suicide, on='state' )

suicide_rate_service.to_csv('Output/suicide_rate_service.csv')

In [48]:
#MH Facilities offering service in Spanish
mh_language = merge_15_16_17_18[['Num_Facilities_2015','LANG_SPAN_2015','Num_Facilities_2016','LANG_SPAN_2016', 'Num_Facilities_2017','LANG_SPAN_2017', 'Num_Facilities_2018','LANG_SPAN_2018' ]]

#Create columns that divide facilities serving spanish speakers by total number of facilities for each year
mh_language.loc[:,'2015'] = (mh_language['LANG_SPAN_2015']/mh_language['Num_Facilities_2015'])*100
mh_language.loc[:,'2016'] = (mh_language['LANG_SPAN_2016']/mh_language['Num_Facilities_2016'])*100
mh_language.loc[:,'2017'] = (mh_language['LANG_SPAN_2017']/mh_language['Num_Facilities_2017'])*100
mh_language.loc[:,'2018'] = (mh_language['LANG_SPAN_2018']/mh_language['Num_Facilities_2018'])*100

mh_language_trends = mh_language[['2015','2016', '2017', '2018']]

#Export dataframe to csv
mh_language_trends.to_csv('Output/mh_language_trends.csv')
mh_language_trends.head()

,2015,2016,2017,2018
state,,,,
Alabama,10.659898,13.989637,14.285714,14.689266
Alaska,10.679612,10.101010,12.790698,13.186813
Arizona,63.212435,62.698413,59.139785,60.821918
Arkansas,13.127413,13.191489,12.844037,15.418502
California,73.932092,75.568182,76.558891,77.203290


In [49]:
#MH Facilities offering services to those who cannot pay for it
mh_payassist = merge_party[['Num_Facilities_2015','PAYASST_2015','Num_Facilities_2016','PAYASST_2016', 'Num_Facilities_2017','PAYASST_2017', 'Num_Facilities_2018','PAYASST_2018', 'Party' ]]

#Create columns that divide facilities offering services to those who cannot pay for its by total number of facilities for each year
mh_payassist.loc[:,'2015'] = (mh_payassist['PAYASST_2015']/mh_payassist['Num_Facilities_2015'])*100
mh_payassist.loc[:,'2016'] = (mh_payassist['PAYASST_2016']/mh_payassist['Num_Facilities_2016'])*100
mh_payassist.loc[:,'2017'] = (mh_payassist['PAYASST_2017']/mh_payassist['Num_Facilities_2017'])*100
mh_payassist.loc[:,'2018'] = (mh_payassist['PAYASST_2018']/mh_payassist['Num_Facilities_2018'])*100

mh_payassist_trends = mh_payassist[['2015','2016', '2017', '2018', 'Party']]
mh_payassist_trends = mh_payassist_trends.sort_values('2016', ascending=True)

#Export dataframe to csv
mh_payassist_trends.to_csv('Output/mh_payassist_trends.csv')

mh_payassist_trends.head()

,2015,2016,2017,2018,Party
state,,,,,
South Carolina,29.545455,28.925620,26.605505,39.047619,R
Wisconsin,31.263858,29.069767,30.714286,31.851852,R
Massachusetts,35.882353,36.283186,33.333333,35.409836,D
Kentucky,33.944954,36.651584,50.480769,46.046512,R
Rhode Island,26.562500,37.096774,37.500000,37.500000,D


In [50]:
#MH Facilities offering services to children age 0-12
mh_child = merge_15_16_17_18[['Num_Facilities_2015','CHILDAD_2015','Num_Facilities_2016','CHILDAD_2016', 'Num_Facilities_2017','CHILDAD_2017', 'Num_Facilities_2018','CHILDAD_2018' ]]

#Create columns that divide facilities serving children age 0-12 by total number of facilities for each year
mh_child.loc[:,'2015'] = (mh_child['CHILDAD_2015']/mh_child['Num_Facilities_2015'])*100
mh_child.loc[:,'2016'] = (mh_child['CHILDAD_2016']/mh_child['Num_Facilities_2016'])*100
mh_child.loc[:,'2017'] = (mh_child['CHILDAD_2017']/mh_child['Num_Facilities_2017'])*100
mh_child.loc[:,'2018'] = (mh_child['CHILDAD_2018']/mh_child['Num_Facilities_2018'])*100

mh_child_trends = mh_child[['2015','2016', '2017', '2018']]

#Export dataframe to csv
mh_child_trends.to_csv('Output/mh_child_trends.csv')

mh_child_trends.head()

,2015,2016,2017,2018
state,,,,
Alabama,43.147208,43.523316,42.857143,44.632768
Alaska,73.786408,75.757576,72.093023,73.626374
Arizona,49.222798,47.883598,48.118280,49.041096
Arkansas,69.884170,71.063830,69.724771,66.519824
California,50.821468,49.659091,49.191686,48.061105


In [51]:
#MH Facilities offering services to adolescents age 13-17
mh_adolescent = merge_15_16_17_18[['Num_Facilities_2015','ADOLES_2015','Num_Facilities_2016','ADOLES_2016','Num_Facilities_2017','ADOLES_2017', 'Num_Facilities_2018','ADOLES_2018' ]]

#Create columns that divide facilities serving adolescents age 13-17 by total number of facilities for each year
mh_adolescent.loc[:,'2015'] = (mh_adolescent['ADOLES_2015']/mh_adolescent['Num_Facilities_2015'])*100
mh_adolescent.loc[:,'2016'] = (mh_adolescent['ADOLES_2016']/mh_adolescent['Num_Facilities_2016'])*100
mh_adolescent.loc[:,'2017'] = (mh_adolescent['ADOLES_2017']/mh_adolescent['Num_Facilities_2017'])*100
mh_adolescent.loc[:,'2018'] = (mh_adolescent['ADOLES_2018']/mh_adolescent['Num_Facilities_2018'])*100

mh_adolescent_trends = mh_adolescent[['2015','2016', '2017', '2018']]

#Export dataframe to csv
mh_adolescent_trends.to_csv('Output/mh_adolescent_trends.csv')

mh_adolescent_trends.head()

,2015,2016,2017,2018
state,,,,
Alabama,45.177665,45.595855,45.142857,46.892655
Alaska,85.436893,87.878788,87.209302,84.615385
Arizona,54.145078,52.116402,52.956989,53.150685
Arkansas,70.656371,70.212766,68.348624,69.603524
California,56.297919,55.000000,55.311778,53.936545


In [52]:
#MH Facilities offering services to young adults age 18-25
mh_yadults = merge_15_16_17_18[['Num_Facilities_2015','YOUNGADULTS_2015','Num_Facilities_2016','YOUNGADULTS_2016','Num_Facilities_2017','YOUNGADULTS_2017', 'Num_Facilities_2018','YOUNGADULTS_2018' ]]

#Create columns that divide facilities serving young adults age 18-25 by total number of facilities for each year
mh_yadults.loc[:,'2015'] = (mh_yadults['YOUNGADULTS_2015']/mh_yadults['Num_Facilities_2015'])*100
mh_yadults.loc[:,'2016'] = (mh_yadults['YOUNGADULTS_2016']/mh_yadults['Num_Facilities_2016'])*100
mh_yadults.loc[:,'2017'] = (mh_yadults['YOUNGADULTS_2017']/mh_yadults['Num_Facilities_2017'])*100
mh_yadults.loc[:,'2018'] = (mh_yadults['YOUNGADULTS_2018']/mh_yadults['Num_Facilities_2018'])*100

mh_yadults_trends = mh_yadults[['2015','2016', '2017', '2018']]

#Export dataframe to csv
mh_yadults_trends.to_csv('Output/mh_yadults_trends.csv')

mh_yadults_trends.head()

,2015,2016,2017,2018
state,,,,
Alabama,83.248731,82.383420,80.571429,80.790960
Alaska,79.611650,81.818182,80.232558,80.219780
Arizona,85.233161,87.037037,85.483871,87.397260
Arkansas,76.447876,74.893617,80.275229,81.057269
California,83.680175,85.227273,85.796767,86.486486


In [53]:
#MH Facilities offering services to adults age 26-64
mh_adults = merge_15_16_17_18[['Num_Facilities_2015','ADULT_2015','Num_Facilities_2016','ADULT_2016','Num_Facilities_2017','ADULT_2017', 'Num_Facilities_2018','ADULT_2018' ]]

#Create columns that divide facilities serving adults age 26-64 by total number of facilities for each year
mh_adults.loc[:,'2015'] = (mh_adults['ADULT_2015']/mh_adults['Num_Facilities_2015'])*100
mh_adults.loc[:,'2016'] = (mh_adults['ADULT_2016']/mh_adults['Num_Facilities_2016'])*100
mh_adults.loc[:,'2017'] = (mh_adults['ADULT_2017']/mh_adults['Num_Facilities_2017'])*100
mh_adults.loc[:,'2018'] = (mh_adults['ADULT_2018']/mh_adults['Num_Facilities_2018'])*100

mh_adults_trends = mh_adults[['2015','2016', '2017', '2018']]

#Export dataframe to csv
mh_adults_trends.to_csv('Output/mh_adults_trends.csv')

mh_adults_trends.head()

,2015,2016,2017,2018
state,,,,
Alabama,85.786802,84.974093,85.714286,85.875706
Alaska,65.048544,66.666667,67.441860,68.131868
Arizona,82.642487,84.656085,83.602151,84.931507
Arkansas,75.289575,73.191489,77.981651,79.295154
California,72.289157,72.500000,72.055427,72.385429


In [54]:
#MH Facilities offering services to seniors age 65+
mh_seniors = merge_15_16_17_18[['Num_Facilities_2015','SENIORS_2015','Num_Facilities_2016','SENIORS_2016','Num_Facilities_2017','SENIORS_2017', 'Num_Facilities_2018','SENIORS_2018' ]]

#Create columns that divide facilities serving adults age 26-64 by total number of facilities for each year
mh_seniors.loc[:,'2015'] = (mh_seniors['SENIORS_2015']/mh_seniors['Num_Facilities_2015'])*100
mh_seniors.loc[:,'2016'] = (mh_seniors['SENIORS_2016']/mh_seniors['Num_Facilities_2016'])*100
mh_seniors.loc[:,'2017'] = (mh_seniors['SENIORS_2017']/mh_seniors['Num_Facilities_2017'])*100
mh_seniors.loc[:,'2018'] = (mh_seniors['SENIORS_2018']/mh_seniors['Num_Facilities_2018'])*100

mh_senior_trends = mh_seniors[['2015','2016', '2017', '2018']]

#Export dataframe to csv
mh_senior_trends.to_csv('Output/mh_senior_trends.csv')

mh_senior_trends.head()

,2015,2016,2017,2018
state,,,,
Alabama,87.309645,84.974093,85.142857,84.745763
Alaska,62.135922,64.646465,65.116279,64.835165
Arizona,78.497409,81.481481,82.258065,82.739726
Arkansas,74.131274,70.638298,75.229358,74.008811
California,59.912377,60.681818,60.969977,61.104583


In [71]:
#Population Data to DataFrame (named test_total_population_data)
file = "Resources/ALL/data/population2010-2019.csv"
total_population_data_file_all = pd.read_csv(file)
total_population_data_file = total_population_data_file_all.copy()
total_population_data_file = total_population_data_file.iloc[5:,4:17]
total_population_data_file = total_population_data_file.reset_index()
total_population_data_file = total_population_data_file.iloc[:,1:]
total_population_data_file = total_population_data_file.rename(columns={"NAME":"state"})
years = ["state"]
for year in range(2015,2019,1):
    total_population_data_file = total_population_data_file.rename(columns={f"POPESTIMATE{year}":f"{year}"})
    years.append(str(year))
total_population_data_file = total_population_data_file.loc[:,years]
total_population_data_file = total_population_data_file.iloc[:51]
total_population_data_file.dtypes
total_population_data_file.head()


,state,2015,2016,2017,2018
0,Alabama,4852347,4863525,4874486,4887681
1,Alaska,737498,741456,739700,735139
2,Arizona,6829676,6941072,7044008,7158024
3,Arkansas,2978048,2989918,3001345,3009733
4,California,38918045,39167117,39358497,39461588


In [73]:
#GDP Data to DataFrame (note:measure in millions of dollars)
file = "Resources/ALL/data/gdp2010-2019.csv"
gdp_data_file_all = pd.read_csv(file)
gdp_data_file = gdp_data_file_all.copy()
num_of_indexes = int(gdp_data_file["GeoName"].count())
indexes = []
gdp_data_file = gdp_data_file.rename(columns={"GeoName":"state"})
for index in range(num_of_indexes):
    if gdp_data_file.loc[index,"Description"] == "All industry total":
        indexes.append(index)
gdp_data_file = gdp_data_file.iloc[indexes]
gdp_data_file = gdp_data_file.iloc[:52]
gdp_data_file = gdp_data_file.reset_index()
gdp_data_file = gdp_data_file.loc[:,years]
for year in years:
    try:
        gdp_data_file[year] = gdp_data_file[year].astype(np.float64)
    except ValueError:
        None

gdp_data_file.to_csv("output/econ_gdp.csv", index=False, header=True)
gdp_data_file.dtypes
gdp_data_file.head()

,state,2015,2016,2017,2018
0,United States *,18224780.0,18715040.0,19519424.0,20580223.0
1,Alabama,200402.7,203829.8,210364.4,221735.5
2,Alaska,50641.6,49363.4,51803.1,54734.1
3,Arizona,297141.2,311091.0,327495.8,348297.1
4,Arkansas,118761.2,120374.8,123383.1,128418.9


In [76]:
#GDP per capita Data to DataFrame
gdppercapita_data_file = gdp_data_file.copy()
gdppercapita_data_file = gdppercapita_data_file.merge(total_population_data_file, on="state")
years_only = years[1:]
for year in years_only:
    gdppercapita_data_file[year] = gdppercapita_data_file[f"{year}_x"]/gdppercapita_data_file[f"{year}_y"]
gdppercapita_data_file = gdppercapita_data_file[years]

gdppercapita_data_file.to_csv("output/econ_gdppercapita.csv", index=False, header=True)
gdppercapita_data_file.dtypes
gdppercapita_data_file.head()

,state,2015,2016,2017,2018
0,Alabama,0.041300,0.041910,0.043156,0.045366
1,Alaska,0.068667,0.066576,0.070033,0.074454
2,Arizona,0.043507,0.044819,0.046493,0.048658
3,Arkansas,0.039879,0.040260,0.041109,0.042668
4,California,0.065619,0.067858,0.071626,0.075966


In [78]:
#Personal Consumption Per Capita to DataFrame (note:measure in dollars)
file = "Resources/ALL/data/personalconsumptionpercapita1997_2018.csv"
personalconsumptionpercapita_data_file_all = pd.read_csv(file)
allpersonalconsumptionpercapita_data_file = personalconsumptionpercapita_data_file_all.copy()
num_of_indexes = int(allpersonalconsumptionpercapita_data_file["GeoName"].count())
indexes = []
allpersonalconsumptionpercapita_data_file = allpersonalconsumptionpercapita_data_file.rename(columns={"GeoName":"state"})
for index in range(num_of_indexes):
    if allpersonalconsumptionpercapita_data_file.loc[index,"Description"] == "Per capita personal consumption expenditures":
        indexes.append(index)
allpersonalconsumptionpercapita_data_file = allpersonalconsumptionpercapita_data_file.iloc[indexes]
allpersonalconsumptionpercapita_data_file = allpersonalconsumptionpercapita_data_file.reset_index()
allpersonalconsumptionpercapita_data_file = allpersonalconsumptionpercapita_data_file.iloc[:52]
allpersonalconsumptionpercapita_data_file = allpersonalconsumptionpercapita_data_file.loc[:,years]
for year in years:
    try:
        gdp_data_file[year] = gdp_data_file[year].astype(np.float64)
    except ValueError:
        None
allpersonalconsumptionpercapita_data_file.to_csv("output/econ_allconsumption.csv", index=False, header=True)
allpersonalconsumptionpercapita_data_file.dtypes
allpersonalconsumptionpercapita_data_file.head()


,state,2015,2016,2017,2018
0,United States,38278.0,39440.0,40922.0,42757.0
1,Alabama,30843.0,31497.0,32527.0,33904.0
2,Alaska,45345.0,46205.0,48053.0,50246.0
3,Arizona,32438.0,33059.0,34471.0,35906.0
4,Arkansas,30596.0,31628.0,32913.0,34095.0


In [80]:
#Health Care Consumption Per Capita to DataFrame (note:measure in dollars)
file = "Resources/ALL/data/personalconsumptionpercapita1997_2018.csv"
personalconsumptionpercapita_data_file_all = pd.read_csv(file)
health_personalconsumptionpercapita_data_file = personalconsumptionpercapita_data_file_all.copy()
num_of_indexes = int(health_personalconsumptionpercapita_data_file["GeoName"].count())
indexes = []
health_personalconsumptionpercapita_data_file = health_personalconsumptionpercapita_data_file.rename(columns={"GeoName":"state"})
for index in range(num_of_indexes):
    if health_personalconsumptionpercapita_data_file.loc[index,"Description"] == "   Health care":
        indexes.append(index)
health_personalconsumptionpercapita_data_file = health_personalconsumptionpercapita_data_file.iloc[indexes]
health_personalconsumptionpercapita_data_file = health_personalconsumptionpercapita_data_file.reset_index()
health_personalconsumptionpercapita_data_file = health_personalconsumptionpercapita_data_file.iloc[:52]
health_personalconsumptionpercapita_data_file = health_personalconsumptionpercapita_data_file.loc[:,years]
for year in years:
    try:
        gdp_data_file[year] = gdp_data_file[year].astype(np.float64)
    except ValueError:
        None

health_personalconsumptionpercapita_data_file.to_csv("output/econ_health_consumption.csv", index=False, header=True)
health_personalconsumptionpercapita_data_file.dtypes
health_personalconsumptionpercapita_data_file.head()


,state,2015,2016,2017,2018
0,United States,6414.0,6686.0,6900.0,7191.0
1,Alabama,5485.0,5619.0,5806.0,6019.0
2,Alaska,9607.0,10020.0,10730.0,11531.0
3,Arizona,5169.0,5329.0,5584.0,5851.0
4,Arkansas,5413.0,5762.0,6048.0,6299.0


In [82]:
#Health Care to Personal Consumption Ratio Per Capita to DataFrame 
health_personal_consumptionratio = health_personalconsumptionpercapita_data_file["state"]
health_personal_consumptionratio = pd.DataFrame(health_personal_consumptionratio)
for year in years:
    try:
        total = allpersonalconsumptionpercapita_data_file[year]
        health = health_personalconsumptionpercapita_data_file[year]
        health_personal_consumptionratio[year] = 100*health/total
    except TypeError:
        None

health_personal_consumptionratio
health_personal_consumptionratio.to_csv("output/econ_health_consumptionratio.csv", index=False, header=True)

health_personal_consumptionratio.head()

,state,2015,2016,2017,2018
0,United States,16.756361,16.952333,16.861346,16.818299
1,Alabama,17.783614,17.839794,17.849786,17.753067
2,Alaska,21.186459,21.685965,22.329511,22.949090
3,Arizona,15.935014,16.119665,16.199124,16.295327
4,Arkansas,17.691855,18.218035,18.375718,18.474850
